In [38]:
from pytube import YouTube
import pandas as pd
from moviepy.editor import *
import os
import pytube
from pytube import YouTube
from pytube.exceptions import VideoUnavailable
from pytube.exceptions import VideoPrivate, AgeRestrictedError,LiveStreamError, RecordingUnavailable, MembersOnly, VideoRegionBlocked
import pandas as pd
import numpy as np
import pytube.exceptions
import matplotlib.pyplot as plt
from google.cloud import storage

In [2]:
#getting the csv file and dropping videos, which end_time is more than 300
def get_data(x):
    #reading the csv
    df = pd.read_csv("/Users/philkolling/Downloads/kinetics700_2020/train.csv")
    df =df.copy().head(x)
    #dropping the videos, which end_time is higher than 300
    df =df.drop(df.loc[df['time_end']>300].index).reset_index(drop=True)
    return df

In [3]:
def remove_long_videos(df):
    test = df.copy()
    #count is just for testing
    count = 0
    #list of all the lengths in order to build the new column
    lengths = []
    #iterating over through all the videos
    for video in range(0,len(test)):
        url = "https://www.youtube.com/watch?v="+test.youtube_id[video]
        yt = YouTube(url)
        #handling error messages
        try:
            stream = yt.streams
        except (KeyError,VideoPrivate,VideoUnavailable,AgeRestrictedError,LiveStreamError, RecordingUnavailable, MembersOnly, VideoRegionBlocked):
            test.drop(video,inplace=True)
            continue
        #drop video if longer than 5min
        if yt.length > 300:
            test.drop(video,inplace=True)
        #if video is already clipped, but end_time is higher than lenght of the video, we set new start and end points
        elif yt.length == 10:
            test.time_start[video] = 0
            test.time_end[video] = yt.length
            length = yt.length
            lengths.append(length)
        #if time_end greater than the length and it's not clipped we drop it
        elif yt.length < test.time_end[video]:
            test.drop(video,inplace=True)
        #appending the lengths to the list     
        else:
            length = yt.length
            lengths.append(length)
        count = count +1
        print(count)
    #creating the new column    
    test["lenghts"] = lengths
    #returning the new dataframe
    return test.reset_index(drop=True)        
                
        

In [5]:
def get_clean_data(x):
    df = get_data(x)
    df = remove_long_videos(df)
    return df

In [9]:
#tmp=get_clean_data(20)
tmp

,label,youtube_id,time_start,time_end,split,lenghts
0,clay pottery making,---0dWlqevI,19,29,train,30
1,news anchoring,---aQ-tA5_A,9,19,train,206
2,using bagging machine,---j12rm3WI,14,24,train,153
3,javelin throw,--07WQ2iBlw,0,10,train,10
4,climbing a rope,--0NTAs-fA0,29,39,train,39
5,sipping cup,--0l35AkU34,68,78,train,90
6,flipping pancake,--33Lscn6sk,0,10,train,10
7,eating spaghetti,--3ouPhoy2A,20,30,train,43
8,dribbling basketball,--4-0ihtnBU,58,68,train,156
9,playing tennis,--56QUhyDQM,185,195,train,205


In [39]:
#download youtube video to folder, filename= youtube_id
path = "/Users/philkolling/Downloads/YoutubeVideos" #path has to be edited
path_test = "/Users/philkolling/Downloads/EditedVideos" #path_test has to be edited
client = storage.Client()
bucket = client.get_bucket('737-human-action-recognition-bucket')
def videoDownload(df): 
    URL = "https://www.youtube.com/watch?v="
    for i in range(0,len(df)):
        # creating the URL link
        video = YouTube(URL + df.youtube_id[i])
        #downloading the video in a local folder, name of the video is label/youtube_id
        video.streams.get_by_itag(18).download(path,filename=f"{df.label[i]}_{df.youtube_id[i]}.mp4")
        
        #creating the path to the video to clip it
        source_path = os.path.join(path,f'{df.label[i]}_{df.youtube_id[i]}.mp4')
        clip = VideoFileClip(source_path)
        #defining the frames per second
        fps = 10
        #creating the subclips with start and end from the csv
        subclip =clip.subclip(df.time_start[i],df.time_end[i])
        #cutting out the audio
        subclip = subclip.without_audio()
        
        #save the cliped file to another folder 
        output_path = os.path.join(path_test,f'{df.label[i]}_{df.youtube_id[i]}.mp4')
        subclip.write_videofile(output_path,fps)
        
        #remove original youtube video
        os.remove(f"{path}/{df.label[i]}_{df.youtube_id[i]}.mp4")
        
        #getting the actual filename we want to upload
        file = os.listdir(f'{path_test}')[0]
        
        #cretinga blob in order to upload it to the right folder based on the label of the video
        blob = bucket.blob('train_data/df.label[video]/file')
        blob.upload_from_filename('file')
        
        #remove the cliped video from the local computer
        os.remove(f"{path_test}/{df.label[i]}_{df.youtube_id[i]}.mp4")
        
        
        
        
        

Forbidden: 403 GET https://storage.googleapis.com/storage/v1/b/737-human-action-recognition-bucket?projection=noAcl&prettyPrint=false: philsaccount@wagon-bootcamp-332120.iam.gserviceaccount.com does not have storage.buckets.get access to the Google Cloud Storage bucket.

In [32]:
#videoDownload(tmp)

In [ ]:

    #take youtube video from local file and cut it to a 10sec video to upload it to gcp
    #Steps:
        #download youtube video with clean df
        #save in local directory
        #load file from local directory
        #cut video with movie.py to 10sec
        #upload to gcp in the right folder based on label
    #Questions:
        #filename?
        #tell the computer which folder it should take

'planing wood_--6zxbYq5M4.mp4'